# Guided Project: Predicting House Sale Prices 

## Introduction 

In the Linear Regression for Machine Learning course, I started building an intuition for model based learning and saw how linear regression works. I also saw the two different approaches to model fitting (Gradient Descent and Ordinary Least Squares) and some cleaning, transforming, and selecting of certain features to help improve a Linear Regression Model. I will work with the housing data for the city of Ames, Iowa from the years 2006-2010. The pipeline of functions that will allow us to iterate on different models are: 

   1) train
   
   2) transform_features()
   
   3) select_features()
   
   4) train_and_test()
   
   5) See both MSE and Average MSE values. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


In [2]:
data = pd.read_csv('AmesHousing.tsv', delimiter = '\t')

def transform_features(data):
    return data

def select_features(data): 
    return data[['Gr Liv Area', 'SalePrice']]

def train_test(data):
    train = data[:1460]
    test = data[1460:]
    numeric_train = train.select_dtypes(include = ['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    attributes = numeric_train.columns.drop('SalePrice')
    
    lr = linear_model.LinearRegression()
    lr.fit(train[attributes], train['SalePrice'])
    prediction = lr.predict(test[attributes])
    mse = mean_squared_error(prediction, test['SalePrice'])
    rmse = np.sqrt(mse)
    return rmse
    
transformed_feats = transform_features(data)
selected_feats = select_features(transformed_feats)
rmse = train_test(selected_feats)
print(rmse)
    

57088.25161263909


## Feature Engineering 

After creating those functions, I can now start removing features with missing values, analyze some categorical features, and transform both text and numericla columns. I will also update the transform_features function so that for any column from the data frame with more than 25% of missing values is not included in the analysis. The goal of the transform_features function is to mainly: 

   1) Remove features that we don't want to use in the model, based on 
      the numnber of missing values or data leakage. 
   
   2) Transform features into the proper format. 
   
   3) Create new features by incorporating other features. 
   


In [3]:
#Remove missing values
missing_values = data.isnull().sum()
drop_missing_column = missing_values[(missing_values > len(data)/20)].sort_values()
data = data.drop(drop_missing_column.index, axis = 1)

#Remove missing text
text_missing = data.select_dtypes(include = ['object']).isnull().sum().sort_values(ascending = False)
drop_column_2 = text_missing[text_missing > 0]
data = data.drop(drop_column_2.index, axis = 1)

#Find values to impute to missing values
numeric_missing = data.select_dtypes(include = ['float', 'int']).isnull().sum()
numeric_impute = numeric_missing[(numeric_missing < len(data)/20) & (numeric_missing >0)].sort_values()
numeric_impute

#Fill in missing values
imputing_value = data[numeric_impute.index].mode().to_dict(orient = 'records')[0]
imputing_value

data = data.fillna(imputing_value)
data.isnull().sum().value_counts()



0    64
dtype: int64

In [4]:
#Update all functions specifically the transform_features() function

def transform_features(data):
    #Remove missing values
    missing_values = data.isnull().sum()
    drop_missing_column = missing_values[(missing_values > len(data)/20)].sort_values()
    data = data.drop(drop_missing_column.index, axis = 1)

    #Remove missing text
    text_missing = data.select_dtypes(include = ['object']).isnull().sum().sort_values(ascending = False)
    drop_column_2 = text_missing[text_missing > 0]
    data = data.drop(drop_column_2.index, axis = 1)

    #Find values to impute to missing values
    numeric_missing = data.select_dtypes(include = ['float', 'int']).isnull().sum()
    numeric_impute = numeric_missing[(numeric_missing < len(data)/20) & (numeric_missing >0)].sort_values()
    numeric_impute

    #Fill in missing values
    imputing_value = data[numeric_impute.index].mode().to_dict(orient = 'records')[0]
    imputing_value

    data = data.fillna(imputing_value)
    data.isnull().sum().value_counts()

    #Start feature engineering
    year_sold = data['Yr Sold'] - data['Year Built']
    year_sold[year_sold < 0 ]

    year_remodeled = data['Yr Sold'] - data['Year Remod/Add']
    year_remodeled[year_remodeled < 0]

    data['Years Before Sale'] = year_sold
    data['Years Since Remod'] = year_remodeled

    #Drop these row numbers
    data = data.drop([1702, 2180, 2181 ], axis = 0)

    #Drop these columns 
    data = data.drop(['PID', 'Order','Mo Sold', 'Sale Condition', 'Sale Type', 'Year Built', 'Year Remod/Add'], axis = 1)
    return data


def select_features(data): 
    return data[['Gr Liv Area', 'SalePrice']]

def train_test(data):
    train = data[:1460]
    test = data[1460:]
    numeric_train = train.select_dtypes(include = ['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    attributes = numeric_train.columns.drop('SalePrice')
    
    lr = linear_model.LinearRegression()
    lr.fit(train[attributes], train['SalePrice'])
    prediction = lr.predict(test[attributes])
    mse = mean_squared_error(prediction, test['SalePrice'])
    rmse = np.sqrt(mse)
    return rmse
    
transformed_feats = transform_features(data)
selected_feats = select_features(transformed_feats)
rmse = train_test(selected_feats)
print(rmse)


55275.36731241307


## Feature Selection 

In [5]:
#Calculate the correlations of the features to the SalePrice column
numerical_dataframe = transformed_feats.select_dtypes(include=['int', 'float'])
numerical_dataframe.head()
correlation = numerical_dataframe.corr()['SalePrice'].abs().sort_values()
correlation

#Filter out all correlations > 0.4
correlation[correlation > 0.4]

#Drop Columns with correlation < 0.4
transformed_feats = transformed_feats.drop(correlation[correlation < 0.4].index, axis = 1)

#Categorical Columns 
categorical_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

#See which categorical columns I keep

updated_categorical_features = []

for col in categorical_features:
    if col in transformed_feats.columns: 
        updated_categorical_features.append(col)
        
unique_counts = transformed_feats[updated_categorical_features].apply(lambda col: len(col.value_counts())).sort_values()
non_unique_columns = unique_counts[unique_counts > 10].index
transformed_data = transformed_feats.drop(non_unique_columns, axis = 1)

text_columns = transformed_data.select_dtypes(include = ['object'])
for col in text_columns: 
    transformed_data[col] = transformed_data[col].astype('category')
    
transformed_data = pd.concat([transformed_data, pd.get_dummies(transformed_data.select_dtypes(include = ['category']))], axis = 1).drop(text_columns, axis = 1)




## Train and Test

In [6]:
#Update all functions specifically the transform_features() function

def transform_features(data):
    #Remove missing values
    missing_values = data.isnull().sum()
    drop_missing_column = missing_values[(missing_values > len(data)/20)].sort_values()
    data = data.drop(drop_missing_column.index, axis = 1)

    #Remove missing text
    text_missing = data.select_dtypes(include = ['object']).isnull().sum().sort_values(ascending = False)
    drop_column_2 = text_missing[text_missing > 0]
    data = data.drop(drop_column_2.index, axis = 1)

    #Find values to impute to missing values
    numeric_missing = data.select_dtypes(include = ['float', 'int']).isnull().sum()
    numeric_impute = numeric_missing[(numeric_missing < len(data)/20) & (numeric_missing >0)].sort_values()
    numeric_impute

    #Fill in missing values
    imputing_value = data[numeric_impute.index].mode().to_dict(orient = 'records')[0]
    imputing_value

    data = data.fillna(imputing_value)
    data.isnull().sum().value_counts()

    #Start feature engineering
    year_sold = data['Yr Sold'] - data['Year Built']
    year_sold[year_sold < 0 ]

    year_remodeled = data['Yr Sold'] - data['Year Remod/Add']
    year_remodeled[year_remodeled < 0]

    data['Years Before Sale'] = year_sold
    data['Years Since Remod'] = year_remodeled

    #Drop these row numbers
    data = data.drop([1702, 2180, 2181 ], axis = 0)

    #Drop these columns 
    data = data.drop(['PID', 'Order','Mo Sold', 'Sale Condition', 'Sale Type', 'Year Built', 'Year Remod/Add'], axis = 1)
    return data


def select_features(data, coeff_threshold = 0.4, unique_threshold = 10): 
    #Calculate the correlations of the features to the SalePrice column
    numerical_dataframe = data.select_dtypes(include=['int', 'float'])
    numerical_dataframe.head()
    correlation = numerical_dataframe.corr()['SalePrice'].abs().sort_values()
    data = data.drop(correlation[correlation < coeff_threshold].index, axis = 1)

    #Categorical Columns 
    categorical_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

    #See which categorical columns I keep

    updated_categorical_features = []

    for col in categorical_features:
        if col in transformed_feats.columns: 
            updated_categorical_features.append(col)
        
    unique_counts = data[updated_categorical_features].apply(lambda col: len(col.value_counts())).sort_values()
    non_unique_columns = unique_counts[unique_counts > 10].index
    data = data.drop(non_unique_columns, axis = 1)

    text_columns = data.select_dtypes(include = ['object'])
    for col in text_columns: 
        data[col] = data[col].astype('category')
    
    data = pd.concat([data, pd.get_dummies(data.select_dtypes(include = ['category']))], axis = 1).drop(text_columns, axis = 1)
    return data


def train_and_test(data, k = 0):
    numeric_data = data.select_dtypes(include = ['int', 'float'])
    features = numeric_data.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    
    if k==0:
        train = data[:1460]
        test = data[1460:]
        lr.fit(train[attributes], train['SalePrice'])
        prediction = lr.predict(test[attributes])
        mse = mean_squared_error(prediction, test['SalePrice'])
        rmse = np.sqrt(mse)
        return rmse
 
    if k==1: 
        shuffled_data = data.sample(frac = 1, )
        train = data[:1460]
        test =  data[1460:]
        lr.fit(train[attributes], train['SalePrice'])
        prediction = lr.predict(test[attributes])
        mse = mean_squared_error(prediction, test['SalePrice'])
        rmse = np.sqrt(mse)
        
        lr.fit(test[features], test['SalePrice'])
        prediction2 = lr.predict(train[features])
        mse2 = mean_squared_error(prediction2, train['SalePrice'])
        rmse2 = np.sqrt(mse2)
        avg_rmse = np.mean(rmse, rmse2)
        return avg_rmse
    
    else: 
        kf = KFold(n_splits = k, shuffle = True)
        rmse_values = []
        for train_index, test_index in kf.split(data):
            train = data.iloc[train_index]
            test = data.iloc[test_index]
            lr.fit(train[features], train['SalePrice'])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(predictions, test['SalePrice'])
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse
    
data = transform_features(data)
data = select_features(data)
rmse = train_and_test(data, k = 4)
print(rmse)



[40244.20587914921, 31895.125100315996, 29222.710164768854, 30184.847827492475]
32886.72224293163
